# Implementing Sauerland's algorithm using the pointed-set monad

In [34]:
import Data.List (nub)

newtype Var = Var Char
    deriving (Eq, Ord)
    
data Expr = Variable Var
          | Negation Expr
          | Conjunction Expr Expr
          | Disjunction Expr Expr
          | L Expr Expr
          | R Expr Expr
          deriving Eq
          
mkVar :: Char -> Expr
mkVar = Variable . Var

instance Show Var where
    show (Var v) = [v]

In [35]:
showBinaryConnective :: (Expr -> String) -> String -> Expr -> Expr -> String
showBinaryConnective show_ symbol exp1 exp2 =
  '(' : show_ exp1 ++ " " ++ symbol ++ " " ++ show_ exp2 ++ ")"

showBC :: String -> Expr -> Expr -> String
showBC = showBinaryConnective show
          
instance Show Expr where
  show (Variable      name)      = show name
  show (Negation      expr)      = '~' : show expr
  show (Conjunction   exp1 exp2) = showBC "&&" exp1 exp2
  show (Disjunction   exp1 exp2) = showBC "||" exp1 exp2
  show (L   exp1 exp2) = showBC "L" exp1 exp2
  show (R   exp1 exp2) = showBC "R" exp1 exp2

Chung-chieh Shan's *pointed-set monad*, taken from http://conway.rutgers.edu/~ccshan/wiki/blog/posts/Pointed_set/

In [36]:
import Control.Monad (liftM,ap)

data Pointed a = Pointed a [a]
  deriving (Eq, Ord, Show, Read)

toList (Pointed x xs) = x:xs

instance Functor Pointed where
  fmap = liftM

instance Applicative Pointed where
  pure = return
  (<*>) = ap
  
instance Monad Pointed where
  return x = Pointed x []
  Pointed x xs >>= k = Pointed y (ys ++ (xs >>= toList.k))
    where Pointed y ys = k x

In [37]:
binaryConnectives :: [Expr -> Expr -> Expr]
binaryConnectives = [Conjunction, Disjunction, R, L]

alts :: (Expr -> Expr -> Expr) -> Pointed (Expr -> Expr -> Expr)
alts op = Pointed op binaryConnectives

removeDupAlts :: Pointed Expr -> Pointed Expr
removeDupAlts (Pointed expr alts) = Pointed expr (nub alts)

In [38]:
(alts Disjunction) <*> ((alts Disjunction) <*> pure (mkVar 'a') <*> pure (mkVar 'b')) <*> pure (mkVar 'c')

Line 1: Redundant bracket
Found:
(alts Disjunction) <*>
  ((alts Disjunction) <*> pure (mkVar 'a') <*> pure (mkVar 'b'))
Why not:
alts Disjunction <*>
  ((alts Disjunction) <*> pure (mkVar 'a') <*> pure (mkVar 'b'))Line 1: Redundant bracket
Found:
(alts Disjunction) <*> pure (mkVar 'a')
Why not:
alts Disjunction <*> pure (mkVar 'a')

Pointed ((a || b) || c) [((a && b) || c),((a || b) || c),((a R b) || c),((a L b) || c),((a || b) && c),((a && b) && c),((a || b) && c),((a R b) && c),((a L b) && c),((a || b) || c),((a && b) || c),((a || b) || c),((a R b) || c),((a L b) || c),((a || b) R c),((a && b) R c),((a || b) R c),((a R b) R c),((a L b) R c),((a || b) L c),((a && b) L c),((a || b) L c),((a R b) L c),((a L b) L c)]